In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/COMP 545 Project

/content/drive/MyDrive/Colab Notebooks/COMP 545 Project


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Feature extraction from input data using convolutional layers.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
from torch.utils.data import Dataset, DataLoader
import torch


df = pd.read_csv('Data/New_data.csv')

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text
inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

# Prepare labels
labels = torch.tensor(df['generated'].values)

# Split data into training and validation
train_inputs, val_inputs, train_labels, val_labels = train_test_split(inputs.input_ids, labels, test_size=0.1)
train_masks, val_masks, _, _ = train_test_split(inputs.attention_mask, labels, test_size=0.1)

# Create dataset
class TextDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

# Create the DataLoader for our training and validation sets
train_data = TextDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

val_data = TextDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class Ro_Model(nn.Module):
    def __init__(self, num_classes, transformer_config, roberta_model_name='roberta-base'):
        super(Ro_Model, self).__init__()
        # Load pre-trained RoBERTa
        self.roberta = RobertaModel.from_pretrained(roberta_model_name)

        # Attention Mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=transformer_config['d_model'],
            num_heads=transformer_config['nhead'],
            dropout=transformer_config['dropout']
        )

        # Convolutional Layer
        self.conv_layer = nn.Conv1d(
            in_channels=transformer_config['d_model'],
            out_channels=transformer_config['d_model'],
            kernel_size=3,
            padding=1
        )

        # Gating Mechanism (GLU)
        self.gating_layer = nn.Linear(transformer_config['d_model'], transformer_config['d_model'] * 2)

        # Transformer Encoder Configuration
        # Creating encoder layer according to transformer_config, excluding 'num_layers'
        encoder_layer_config = {k: v for k, v in transformer_config.items() if k != 'num_layers'}
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=transformer_config['d_model'],
            nhead=transformer_config['nhead'],
            dim_feedforward=transformer_config['dim_feedforward'],
            dropout=transformer_config['dropout']
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=transformer_config['num_layers'])

        # Transformer Decoder Configuration
        decoder_layer_config = {k: v for k, v in transformer_config.items() if k != 'num_layers'}
        decoder_layers = nn.TransformerDecoderLayer(
            d_model=transformer_config['d_model'],
            nhead=transformer_config['nhead'],
            dim_feedforward=transformer_config['dim_feedforward'],
            dropout=transformer_config['dropout']
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layers, num_layers=transformer_config['num_layers'])

        # Position-wise Feed-Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(transformer_config['d_model'], transformer_config['dim_feedforward']),
            nn.ReLU(),
            nn.Linear(transformer_config['dim_feedforward'], transformer_config['d_model'])
        )

        # Layer Normalization
        self.layer_norm = nn.LayerNorm(transformer_config['d_model'])

        # Classifier
        self.classifier = nn.Linear(transformer_config['d_model'], num_classes)

    def forward(self, input_ids, attention_mask):
        # Get the last hidden states from RoBERTa
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)

        # Apply Layer Normalization to RoBERTa output
        roberta_output = self.layer_norm(roberta_output.last_hidden_state)

        # Attention Mechanism
        attention_output, _ = self.attention(roberta_output, roberta_output, roberta_output)

        # Residual Connection: Add RoBERTa output to attention output
        attention_output = roberta_output + attention_output

        # Convolutional Layer
        conv_output = self.conv_layer(attention_output.permute(0, 2, 1)).permute(0, 2, 1)

        # Residual Connection: Add attention output to convolutional output
        conv_output = attention_output + conv_output

        # Gating Mechanism (GLU)
        gating_output = self.gating_layer(conv_output)
        gates, linear = gating_output.chunk(2, dim=-1)
        gated_output = gates.sigmoid() * linear

        # Transformer Encoder
        encoder_output = self.transformer_encoder(gated_output)

        # Apply Layer Normalization to encoder output
        encoder_output = self.layer_norm(encoder_output)

        # Residual Connection: Add gated output to encoder output
        encoder_output = gated_output + encoder_output

        # Transformer Decoder
        decoder_output = self.transformer_decoder(encoder_output, memory=encoder_output)

        # Apply Layer Normalization to decoder output
        decoder_output = self.layer_norm(decoder_output)

        # Residual Connection: Add encoder output to decoder output
        decoder_output = encoder_output + decoder_output

        # Position-wise Feed-Forward Network
        ffn_output = self.ffn(decoder_output)

        # Apply Layer Normalization to FFN output
        ffn_output = self.layer_norm(ffn_output)

        # Residual Connection: Add decoder output to FFN output
        ffn_output = decoder_output + ffn_output

        cls_embeddings = ffn_output[:, 0, :]

        logits = self.classifier(cls_embeddings)
        return logits

In [ ]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss

torch.cuda.init()

# Initialize model
num_classes = 2
transformer_config = {
    'd_model': 768,
    'nhead': 8,
    'dim_feedforward': 2048,
    'dropout': 0.1,
    'num_layers': 6,
}

model = Ro_Model(num_classes, transformer_config)

model.to(device)  # Move model to GPU

# Define loss function
loss_fn = CrossEntropyLoss()

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ExponentialLR

def calculate_accuracy(outputs, labels):
    preds = torch.argmax(outputs, dim=1)
    correct = (preds == labels).float()
    acc = correct.sum() / len(correct)
    return acc

epochs = 10

# Set the number of gradient accumulation steps
accumulation_steps = 4

# Define the learning rate scheduler
scheduler = ExponentialLR(optimizer, gamma=0.8)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_acc = 0
    # Wrap train_dataloader with tqdm for a progress bar
    train_progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs} Training')
    for batch_idx, batch in enumerate(train_progress_bar, start=1):
        outputs = model(batch['input_ids'].to(device).long(), batch['attention_mask'].to(device).float())
        labels = batch['labels'].to(device).long()
        loss = loss_fn(outputs.float(), labels)
        acc = calculate_accuracy(outputs, labels)

        # Normalize the loss to account for batch accumulation
        loss = loss / accumulation_steps

        # Backward pass
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Accumulate gradients
        if batch_idx % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps
        total_acc += acc.item()
        # Update the progress bar with the current loss
        train_progress_bar.set_postfix({'loss': loss.item() * accumulation_steps, 'acc': acc.item()})

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / len(train_dataloader)
    avg_acc = total_acc / len(train_dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss}, Accuracy: {avg_acc}")

    # Update the learning rate
    scheduler.step()

    # Validation phase
    model.eval()
    total_eval_loss = 0
    total_eval_acc = 0
    # Wrap val_dataloader with tqdm for a progress bar
    val_progress_bar = tqdm(val_dataloader, desc=f'Epoch {epoch+1}/{epochs} Validation')
    with torch.no_grad():
        for batch in val_progress_bar:
            outputs = model(batch['input_ids'].to(device).long(), batch['attention_mask'].to(device).float())
            labels = batch['labels'].to(device).long()
            loss = loss_fn(outputs.float(), labels)
            acc = calculate_accuracy(outputs, labels)
            total_eval_loss += loss.item()
            total_eval_acc += acc.item()
            # Update the progress bar with the current validation loss
            val_progress_bar.set_postfix({'val_loss': loss.item(), 'val_acc': acc.item()})

        print(f"Validation Loss: {total_eval_loss / len(val_dataloader)}, Validation Accuracy: {total_eval_acc / len(val_dataloader)}")

Epoch 1/10 Training:   0%|          | 0/900 [00:00<?, ?it/s]

Epoch 1, Loss: 0.665025527486707, Accuracy: 0.7634722222222222


Epoch 1/10 Validation:   0%|          | 0/100 [00:00<?, ?it/s]

Validation Loss: 0.5792785394657404, Validation Accuracy: 0.82875


Epoch 2/10 Training:   0%|          | 0/900 [00:00<?, ?it/s]

...

Validation Loss: 0.08200477536622203, Validation Accuracy: 0.9775


Epoch 10/10 Training:   0%|          | 0/900 [00:00<?, ?it/s]

Epoch 10, Loss: 0.014459264955658, Accuracy: 0.9972222222222222


Epoch 10/10 Validation:   0%|          | 0/100 [00:00<?, ?it/s]

Validation Loss: 0.18063287055760158, Validation Accuracy: 0.975
